In [1]:
# Trantormer는 encoder -> decoder
# encoder 를 이용해서 만든 언어모델 BERT : 감성분류, 스팸, 키워드 추출, 유사도 측정 --> 추출
# decoder 를 이용해서 만든 언어모델 GPT : 언어 추론 요약, QA 챗본 --> 생성


In [2]:
# Bert가 잘하는 것 : 분류, 빈칸 추론, 문장 임베딩

In [2]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [13]:
from transformers import BertForMaskedLM
from transformers import GPT2LMHeadModel
model = BertForMaskedLM.from_pretrained('skt/kobert-base-v1')
model.eval()

Some weights of BertForMaskedLM were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [15]:

from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained("klue/bert-base")
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

c:\Users\31799\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\31799\.cache\huggingface\hub\models--klue--bert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaske

In [17]:
# 1. 상식 추론
import torch
text = '옛날옛적에 호랑이가 [MASK]에 살았어요.'
inputs = tokenizer(text,return_tensors='pt')
mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]
# 추론
with torch.no_grad():
  outputs = model(**inputs)
predictions = outputs.logits
print(predictions[0,mask_token_index,:])
masked_prediction = predictions[0,mask_token_index,:].topk(5)
for i, index_t in enumerate(masked_prediction.indices[0]):
  index = index_t.item()
  print(tokenizer.decode([index]))

tensor([[-5.6930,  8.9020, -4.5985,  ..., -5.3556, -3.0629, -2.7361]])
우리나라
산
[UNK]
서울
한반도


In [19]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2',
bos_token = '<s>', eos_token =  '</s>', unk_token = '<unk>',
pad_token = '<pad>', mask_token = '<mask>')
tokenizer.tokenize('안녕하세요. 한국어 GPT-2 입니다.')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕', '하', '세', '요.', '▁한국어', '▁G', 'P', 'T', '-2', '▁입', '니다.']

In [24]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.eval()
# 프롬프트 엔지니어링
prompt = f'{text}\n\한줄 요약 : '
# 인코딩
outputs = tokenizer(prompt, return_tensors='pt')
# 모델 추론(생성)
with torch.no_grad():
    generated_ids = model.generate(
        max_length=256,
        repetition_penalty=2.0,
        pad_token_id = tokenizer.pad_token_id,
        eos_token_id = tokenizer.eos_token_id,
        bos_token_id = tokenizer.bos_token_id,
        use_cache=True,
        do_sample=True,
        top_k=50,
        temperature=0.7,
    )
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

이자, ’파워풀한 팀플레이’는 지난해 5월 KIBA ‘월드 오브 워크래프트’에서 보여준 열정과 역량이 더욱 빛날 것이라고 밝혔다.
오대엽 원장은 “지난해에 이어 올해도 두 번째 출전하는 WCG는 월드 시리즈와 달리 국가대표 선발전에 국한되지 않고 모든 연령층의 게이머들과 함께 즐길 수 있는 대회로 자리매김할 것”이라며 “이번 대회를 통해 한국 대표팀의 선전과 더불어 국내 e스포츠 팬들이 한층 더 발전된 플레이를 경험했으면 한다”고 말했다.
KeSPA 측은 오는 6월까지 시즌을 진행하며 우승은 물론 준우승과 3위 입상자는 별도로 시상하고 총상금 1,000만원을 수여한다. 경기도문화의전당은 26일 성남시 분당구 야탑동 '경기도문화원'의 개관을 기념, 이달 말까지 입장권 할인행사를 갖는다고 27일 발표했다.
경기문화재단은 이번 행사를 위해 경기지역 19개 문화원의 입주자 총 8000여명을 대상으로 공연·체험 등을 무료로 진행하고 있다.
주말 현장 신청도 가능하다.
성남시립도서관에서는 다음 달 1일부터 9월 28일까지, 서울 도봉도서관은 10월 30일부터 12월 11일까지로 5일간 각각 휴관한다.
또 시흥서라벌아트센터에서 열리는 문학콘텐츠 영화제에도 관람객들을 초대하기
